In [ ]:
#https://github.com/cs-chan/ArtGAN/tree/master/WikiArt%20Dataset

#download data

#unzip data

In [ ]:
#load data

from pathlib import Path

data_dir=Path('./data/wikiart/')



In [2]:
# load data

from pathlib import Path

data_dir = Path('./data/wikiart/')


# count images
cnt = 0
for dir in data_dir.iterdir():
    if dir.is_dir():
        for file in dir.iterdir():
            if str(file).endswith('jpg'):
                cnt+=1
cnt


# loop over all images
#    create caption from directory and file names
#    check image size
#        if any dimension is < 64 -> drop
#        if any dimension is > 512 -> resize
#


81446

In [5]:
from PIL import Image
from tqdm import tqdm
for style_dir in tqdm(data_dir.iterdir()):
    if style_dir.is_dir():
        for file in style_dir.iterdir():
            if str(file).endswith('jpg'):
                #load image
                img = Image.open(file)
                #check size
                height, width = img.size
                if height < 64 or width < 64:
                    file.unlink()
                    continue
                elif height > 512:
                    #resize to 512 and keep aspect ratio
                    img = img.resize((512, int(512*height/width)), Image.ANTIALIAS)
                    #save image as resized
                    img.save('resized_'+file.name)
                elif width > 512:
                    #resize to 512 and keep aspect ratio
                    img = img.resize((int(512*width/height), 512), Image.ANTIALIAS)
                    #save image as resized
                    img.save('resized_'+file.name)
                else:    
                    pass
                #create caption
                caption = style_dir.name + ' ' + file.name
                #print(caption)

#resize image while keeping aspect ratio
#https://stackoverflow.com/questions/44650888/resize-image-without-distortion-using-opencv-python
        

2it [00:58, 29.02s/it]


KeyboardInterrupt: 

In [ ]:
#captions from directories and filenames


caption=f"{style} artwork by {artist_name} called {artwork_name}. Created in {creation_year}."

In [ ]:
#process data

In [ ]:
#